In [62]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import csv
import json 
import pickle
import gc

In [37]:
file = pd.read_csv("process/process_file_progress_0.5.csv", header=None)
file.head()

,0,1,2,3,4,5
0,TIF,37,0.0000,0.000000,78.269997,78.279999
1,CMG,38,0.4939,0.401648,400.500000,402.940002
2,DIS,38,0.2960,0.219123,109.559998,109.000000
3,EBAY,38,-0.4215,0.328668,32.450001,33.250000
4,FB,38,-0.2732,0.200821,132.600006,134.199997


In [38]:
file.columns = ['comp','daynum','sent_score','sent_magn','open','close']

In [22]:
company_group = file.groupby(['comp'], as_index=False).apply(pd.DataFrame.sort_values, ['daynum']).reset_index()
company_group = company_group.drop(columns = ['level_0','level_1'])
company_group.head()

,comp,daynum,sent_score,sent_magn,open,close
0,A,74,0.0000,0.000000,53.430000,53.049999
1,A,80,0.3182,0.237337,52.790001,53.130001
2,A,142,0.0000,0.000000,59.799999,58.660000
3,A,198,0.7003,0.693686,61.430000,61.209999
4,A,200,0.2732,0.200821,61.450001,60.980000


In [24]:
day_sorted = file.sort_values(['daynum'])
day_sorted.head()

,comp,daynum,sent_score,sent_magn,open,close
0,TIF,37,0.0000,0.000000,78.269997,78.279999
1,CMG,38,0.4939,0.401648,400.500000,402.940002
2,DIS,38,0.2960,0.219123,109.559998,109.000000
3,EBAY,38,-0.4215,0.328668,32.450001,33.250000
4,FB,38,-0.2732,0.200821,132.600006,134.199997


In [46]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
new_file = encoder.fit_transform(file.iloc[:,0]).toarray()
new_file.shape

ValueError: Expected 2D array, got 1D array instead:
array=['TIF' 'CMG' 'DIS' ... 'V' 'WMT' 'XOM'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [51]:
np.array(file.iloc[:,0]).reshape(1,-1)

array([['TIF', 'CMG', 'DIS', ..., 'V', 'WMT', 'XOM']], dtype=object)

In [58]:
from sklearn import preprocessing
with open('process/process_file_progress_0.5.csv') as csv_file:
    reader = csv.reader(csv_file)
    data = list(reader)
data = np.array(data)
one_hot_encoder = preprocessing.OneHotEncoder()

processed_data = one_hot_encoder.fit_transform(data[:, 0:1]).toarray()
processed_data = np.append(processed_data, data[:, 1:6], 1)

processed_data = preprocessing.normalize(processed_data)
np.random.shuffle(processed_data)

In [61]:
processed_data[0,:]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [102]:
model = Sequential()
model.add(tf.keras.layers.Lambda(lambda x:tf.expand_dims(x, axis=-1), input_shape=[None]))
model.add(Conv1D(32, 5, activation="relu"))
model.add(MaxPooling1D(4))
model.add(Conv1D(64, 5, activation="relu"))
model.add(MaxPooling1D(4))
model.add(GlobalAveragePooling1D())
model.add(Dense(512, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))

model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001, momentum=0.9), metrics=['mse'], loss='mse')
model.summary(0)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_18 (Lambda)           (None, None, 1)           0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, None, 32)          192       
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, None, 32)          0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, None, 64)          10304     
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, None, 64)          0         
_________________________________________________________________
global_average_pooling1d_13  (None, 64)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 512)             

In [79]:
np.expand_dims(processed_data, axis=-1).shape

(21286, 450, 1)

In [86]:
from sklearn.model_selection import train_test_split
X=processed_data[:,:-1]
y=processed_data[:,-1]
train_X, test_X, train_Y, test_Y=train_test_split(X,y,test_size=0.3)

In [87]:
train_X.shape

(14900, 449)

In [103]:
history = model.fit(train_X, train_Y, epochs=100, verbose=2)

Train on 14900 samples
Epoch 1/100
14900/14900 - 5s - loss: 0.0408 - mean_squared_error: 0.0408
Epoch 2/100
14900/14900 - 4s - loss: 0.0384 - mean_squared_error: 0.0384
Epoch 3/100
14900/14900 - 4s - loss: 0.0384 - mean_squared_error: 0.0384
Epoch 4/100
14900/14900 - 4s - loss: 0.0384 - mean_squared_error: 0.0384
Epoch 5/100
14900/14900 - 4s - loss: 0.0384 - mean_squared_error: 0.0384
Epoch 6/100


KeyboardInterrupt: 